In [1]:
# multivariate output 1d cnn example
import os
import numpy as np
from numpy import array
from numpy import hstack
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from sklearn.model_selection import train_test_split
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from numpy import loadtxt

lines = loadtxt('combined_trial0.txt', delimiter=",", unpack=False, usecols=(4,7,10,13,16,19,22,25,28,31))
#dataset=lines
#for filename in os.scandir('dataset'):
#    if filename.is_file():
#        print(filename)
#        lines=np.append(lines,loadtxt(filename, delimiter=",", unpack=False, usecols=(4,7,10,13,16,19,22,25,28,31))) 
#        dataset = hstack((dataset,loadtxt(filename, delimiter=",", unpack=False, usecols=(4,7,10,13,16,19,22,25,28,31))))
        #print(lines[0][0])
np.random.shuffle(lines)
print(lines.shape)
#print(dataset.shape)




(223357, 10)


In [2]:
len(lines)

223357

In [3]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	print(len(sequences))
	for i in range(len(sequences)):
		seq_x, seq_y = sequences[i, :7], sequences[i, 7:]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


In [4]:

# horizontally stack columns
#dataset = hstack((in_seq1, in_seq2, out_seq))
#print(dataset)
# choose a number of time steps
n_steps = 3


In [5]:
# convert into input/output
X, y = split_sequences(lines, n_steps)
#X=X[None,:,:]
print(X.shape)
print(y.shape)

223357


(223357, 7)
(223357, 3)


In [6]:
# the dataset knows the number of features, e.g. 2
n_features = X.shape[-1]
print('shape='+str(n_features))

shape=7


In [7]:
visible = Input(shape=(7, 1))
cnn = Conv1D(filters=64, kernel_size=2, activation='relu')(visible)
cnn = MaxPooling1D(pool_size=2)(cnn)
cnn = Flatten()(cnn)
cnn = Dense(50, activation='relu')(cnn)
# define output 1
output1 = Dense(1)(cnn)
# define output 2
output2 = Dense(1)(cnn)
# define output 3
output3 = Dense(1)(cnn)


In [8]:
# tie together
model = Model(inputs=visible, outputs=[output1, output2, output3])
model.compile(optimizer='adam', loss='mse')
print(model.summary())


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 7, 1)]       0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 6, 64)        192         ['input_1[0][0]']                
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 3, 64)        0           ['conv1d[0][0]']                 
                                                                                                  
 flatten (Flatten)              (None, 192)          0           ['max_pooling1d[0][0]']          
                                                                                              

In [9]:

print(y.shape)
y1 = y[:, 0].reshape((y.shape[0], 1))
y2 = y[:, 1].reshape((y.shape[0], 1))
y3 = y[:, 2].reshape((y.shape[0], 1))
print(X.shape)
print(y1.shape)

(223357, 3)
(223357, 7)
(223357, 1)


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .75) # [y1,y2,y3]
# fit model
print('fitting')
print(y_train.shape)
ytr1 = y_train[:, 0].reshape((y_train.shape[0], 1))
ytr2 = y_train[:, 1].reshape((y_train.shape[0], 1))
ytr3 = y_train[:, 2].reshape((y_train.shape[0], 1))
yte1 = y_test[:, 0].reshape((y_test.shape[0], 1))
yte2 = y_test[:, 1].reshape((y_test.shape[0], 1))
yte3 = y_test[:, 2].reshape((y_test.shape[0], 1))

model.fit(X_train, [ytr1,ytr2,ytr3], epochs=20,batch_size=6)

fitting
(167517, 3)
Epoch 1/20
27920/27920 [==============================] - 58s 2ms/step - loss: 0.0011 - dense_1_loss: 2.3540e-04 - dense_2_loss: 6.0217e-04 - dense_3_loss: 2.4862e-04
Epoch 2/20
27920/27920 [==============================] - 51s 2ms/step - loss: 9.5444e-04 - dense_1_loss: 2.0126e-04 - dense_2_loss: 5.2526e-04 - dense_3_loss: 2.2792e-04
Epoch 3/20
27920/27920 [==============================] - 36s 1ms/step - loss: 8.8819e-04 - dense_1_loss: 1.8747e-04 - dense_2_loss: 4.8250e-04 - dense_3_loss: 2.1821e-04
Epoch 4/20
27920/27920 [==============================] - 71s 3ms/step - loss: 8.4832e-04 - dense_1_loss: 1.7901e-04 - dense_2_loss: 4.5874e-04 - dense_3_loss: 2.1057e-04
Epoch 5/20
27920/27920 [==============================] - 82s 3ms/step - loss: 8.1159e-04 - dense_1_loss: 1.6970e-04 - dense_2_loss: 4.3782e-04 - dense_3_loss: 2.0407e-04
Epoch 6/20
27920/27920 [==============================] - 75s 3ms/step - loss: 7.7946e-04 - dense_1_loss: 1.6238e-04 - dense_2_lo

In [11]:

score = model.evaluate(X_test, [yte1,yte2,yte3], verbose = 0) 

print("test loss, test acc:", score)

test loss, test acc: [0.0006523958290927112, 0.00011879111843882129, 0.0003588676918298006, 0.00017473743355367333]


In [12]:
lines2 = loadtxt("trial3_revised11.txt", delimiter=",", unpack=False, usecols=(4,7,10,13,16,19,22,25,28,31))

In [13]:
seq_x2, seq_y2 = lines2[100, :7], lines2[100, 7:]
x2=list()
x2.append(seq_x2)
x2=array(x2)
print(x2.shape)

yhat = model.predict(x2, verbose=0)
print(yhat)
print(seq_y2)

(1, 7)
[array([[-0.13696317]], dtype=float32), array([[-0.16898528]], dtype=float32), array([[-0.19982865]], dtype=float32)]
[ 0.151382 -0.02439  -0.064336]
